In [26]:
import json
import os
import os.path
import pandas as pd

In [27]:
raw_age_at_visit = pd.read_csv(os.path.join('..', 'data', 'csv', 'Age_at_visit.csv'))
raw_u3_score = pd.read_csv(os.path.join('..', 'data', 'csv', 'MDS_UPDRS_Part_III_CAL.csv'))
#Deprecated
#raw_u3_on_off = pd.read_csv(os.path.join('..', 'data', 'csv', 'MDS-UPDRS_Part_III_ON_OFF_Determination___Dosing.csv'))
raw_demographic = pd.read_csv(os.path.join('..', 'data', 'csv', 'Demographics.csv'))
raw_img_info = pd.read_csv(os.path.join('..', 'data', 'T1PD.csv'))
raw_diag = pd.read_csv(os.path.join('..', 'data', 'csv', 'PD_Diagnosis_History.csv'))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15236\3154083320.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_u3_score = pd.read_csv(os.path.join('..', 'data', 'csv', 'MDS_UPDRS_Part_III_CAL.csv'))


In [28]:
def apply_filter(x):
    # MED_ON
    on_score = list(x[x['PDSTATE'] == 'ON']['NP3TOT'])
    if len(on_score) == 0:
        on_score = [None]
    # MED_OFF
    off_score = list(x[x['PDSTATE'] == 'OFF']['NP3TOT'])
    if len(off_score) == 0:
        off_score = [None]
    return pd.Series({'INFODT': list(x['INFODT'])[0], 'NUPDR3OF': off_score[0], 'NUPDR3ON': on_score[0]})

In [29]:
# Keep duplicate index
u3_dup_idx = raw_u3_score.duplicated(subset=['PATNO', 'EVENT_ID'], keep=False)
# Get duplicate records
u3_rec = raw_u3_score[u3_dup_idx][['PATNO', 'EVENT_ID', 'INFODT', 'PDSTATE', 'NP3TOT']].dropna().reset_index(drop=True)
# Generate U3 ON/OFF records by PATNO and EVENT_ID
u3_rec = u3_rec.groupby(['PATNO', 'EVENT_ID']).apply(apply_filter).reset_index().dropna().reset_index(drop=True)
# Get image id
image_meta = raw_img_info.rename(columns={'Image Data ID': 'IMG_ID', 'Subject': 'PATNO', 'Visit': 'EVENT_ID'})
# Merge U3 records and image id
data = pd.merge(u3_rec, image_meta, on=['PATNO', 'EVENT_ID'])[['PATNO', 'EVENT_ID', 'INFODT', 'NUPDR3OF', 'NUPDR3ON', 'IMG_ID']].reset_index(drop=True)

In [30]:
# Get image metadata file path
xmllist = []
for dirpath, dirnames, filenames in os.walk(os.path.join('..', 't1meta')):
    for filename in filenames:
        if dirpath == '..\\t1meta':
            continue
        xmllist.append(os.path.join(dirpath, filename))

In [31]:
# Generate image dataframe by PATNO and IMG_ID
img_path = []
from xml.dom import minidom
for xml in xmllist:
    root = minidom.parse(xml).documentElement
    subject = root.getElementsByTagName('subject')[0].getAttribute('id')
    study = root.getElementsByTagName('study')[0].getAttribute('uid')
    series = root.getElementsByTagName('series')[0].getAttribute('uid')
    image = root.getElementsByTagName('image')[0].getAttribute('uid')
    relative_path = os.path.join(xml.split('\\')[2], xml.split('\\')[3], xml.split('\\')[4], series)
    img_path.append({'PATNO': int(subject), 'IMG_ID': str(image), 'SERIES': str(series), 'IMG_REL_PATH': str(relative_path)})
img_path = pd.DataFrame(img_path)

In [32]:
# Merge image df and main df
data = pd.merge(data, img_path, on=['PATNO', 'IMG_ID'])[['PATNO', 'EVENT_ID', 'INFODT', 'NUPDR3OF', 'NUPDR3ON', 'IMG_ID', 'IMG_REL_PATH']].reset_index(drop=True)

In [33]:
# Merge age_at_visit df and main df to extract age by EVENT_ID
data = pd.merge(data, raw_age_at_visit, on=['PATNO', 'EVENT_ID'])[['PATNO', 'EVENT_ID', 'INFODT', 'NUPDR3OF', 'NUPDR3ON', 'IMG_ID', 'IMG_REL_PATH', 'AGE_AT_VISIT']].reset_index(drop=True)

In [34]:
# Merge demographic df and main df
data = pd.merge(data, raw_demographic.drop(labels=['EVENT_ID', 'INFODT'], axis=1), on=['PATNO'])[['PATNO', 'EVENT_ID', 'INFODT', 'NUPDR3OF', 'NUPDR3ON', 'IMG_ID', 'IMG_REL_PATH', 'AGE_AT_VISIT', 'SEX', 'ORIG_ENTRY']].reset_index(drop=True)

In [35]:
# Duration calculation function
def get_duration(rec):
    visit = rec.EVENT_ID
    id = rec.PATNO
    visit = rec.INFODT
    diag = raw_diag[raw_diag['PATNO'] == id]['PDDXDT'].iloc[0]
    visit = visit.split('/')
    diag = diag.split('/')
    return 12 * (int(visit[1]) - int(diag[1])) + int(visit[0]) - int(diag[0])

In [36]:
# Calculate score
data['SCORE'] = (data['NUPDR3OF'] - data['NUPDR3ON']) / data['NUPDR3OF']
# Calculate duration
data['DURATION'] = data.apply(get_duration, axis=1)
# Calculate categories
data['CAT'] = 1 * (data['SCORE'] >= 0.3)

In [37]:
data.describe()

,PATNO,NUPDR3OF,NUPDR3ON,AGE_AT_VISIT,SEX,SCORE,DURATION,CAT
count,363.000000,363.000000,363.00000,363.000000,363.000000,363.000000,363.000000,363.000000
mean,28557.184573,28.363636,19.07438,64.442424,0.641873,0.327477,46.732782,0.523416
std,28588.427233,12.299665,11.24501,9.103962,0.480111,0.244642,21.599511,0.500141
min,3107.000000,2.000000,1.00000,35.100000,0.000000,-0.272727,2.000000,0.000000
25%,3558.000000,19.000000,11.00000,58.400000,0.000000,0.156725,30.000000,0.000000
50%,4083.000000,27.000000,18.00000,65.500000,1.000000,0.312500,51.000000,1.000000
75%,50173.500000,36.000000,25.00000,71.300000,1.000000,0.492424,57.000000,1.000000
max,149511.000000,62.000000,64.00000,86.300000,1.000000,0.956522,112.000000,1.000000


In [38]:
# 以下为影像文件路径相关

In [39]:
# Add root
data['IMG_REL_PATH'] = '..\\t1raw\\' + data['IMG_REL_PATH']

In [40]:
# Extract nii file by path
import shutil
def move_nii(rec):
    orig_path = rec.IMG_REL_PATH
    orig_path = os.path.join(orig_path, os.listdir(rec.IMG_REL_PATH)[0])
    dest_path = os.path.join('..', 't1', str(rec.PATNO)+str(rec.EVENT_ID)+str(rec.IMG_ID))
    #os.mkdir(dest_path)
    dest_path = os.path.join(dest_path, 't1.nii')
    #shutil.copyfile(orig_path, dest_path)
    return dest_path

In [41]:
data['NII_PATH'] = data.apply(move_nii, axis=1)

In [42]:
# Remove 't1.nii' to get IMG_ROOT
data['IMG_ROOT'] = data['NII_PATH'].apply(lambda x: x[:-6])

In [43]:
# Matlab code: M1Segmentation.m

In [44]:
# Preprocessed image path
data['T1_MNI_PATH'] = data['IMG_ROOT'] + '\\mri\\wmt1.nii'
data['T1_GM_PATH'] = data['IMG_ROOT'] + '\\mri\\mwp1t1.nii'

In [45]:
# Matlab code: M2Smooth.m

In [46]:
data['T1_SGM_PATH'] = data['IMG_ROOT'] + '\\mri\\smwp1t1.nii'

In [47]:
data['KEY'] = data['PATNO'].astype(str) + data['EVENT_ID'] + data['IMG_ID']

In [48]:
# LEDD extraction
# ???
raw_ledd = pd.read_csv(os.path.join('..', 'data', 'csv', 'LEDD_Concomitant_Medication_Log.csv'))

In [49]:
# IQR check
iqr_list = []
from xml.dom import minidom
report_list = list(data['IMG_ROOT']+'\\report\\cat_t1.xml')
for report in report_list:
    root = minidom.parse(report).documentElement
    iqr_str = root.getElementsByTagName('catlog')[0].getElementsByTagName('item')[-5].childNodes[0].data
    iqr_str = iqr_str.split(' ')[4][:-1]
    iqr_list.append({'IQR': float(iqr_str)})
iqr_list = pd.DataFrame(iqr_list)
data = pd.concat([data, iqr_list], axis=1)
#data = data[data['IQR'] >= 70]

In [50]:
# Write to json file
data_json = data.to_dict(orient='records')
with open('data.json', 'w+') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

In [51]:
data

,PATNO,EVENT_ID,INFODT,NUPDR3OF,NUPDR3ON,IMG_ID,IMG_REL_PATH,AGE_AT_VISIT,SEX,ORIG_ENTRY,SCORE,DURATION,CAT,NII_PATH,IMG_ROOT,T1_MNI_PATH,T1_GM_PATH,T1_SGM_PATH,KEY,IQR
0,3107,V04,03/2012,2.0,2.0,I296431,..\t1raw\3107\MPRAGE_GRAPPA\2012-03-28_10_35_2...,70.6,1,03/2011,0.000000,14,0,..\t1\3107V04I296431\t1.nii,..\t1\3107V04I296431\,..\t1\3107V04I296431\\mri\wmt1.nii,..\t1\3107V04I296431\\mri\mwp1t1.nii,..\t1\3107V04I296431\\mri\smwp1t1.nii,3107V04I296431,78.43
1,3107,V06,05/2013,18.0,6.0,I378218,..\t1raw\3107\MPRAGE_GRAPPA\2013-05-15_10_04_1...,71.7,1,03/2011,0.666667,28,1,..\t1\3107V06I378218\t1.nii,..\t1\3107V06I378218\,..\t1\3107V06I378218\\mri\wmt1.nii,..\t1\3107V06I378218\\mri\mwp1t1.nii,..\t1\3107V06I378218\\mri\smwp1t1.nii,3107V06I378218,78.22
2,3107,V10,05/2015,19.0,6.0,I498876,..\t1raw\3107\Sag_MPRAGE_GRAPPA\2015-05-08_09_...,73.7,1,03/2011,0.684211,52,1,..\t1\3107V10I498876\t1.nii,..\t1\3107V10I498876\,..\t1\3107V10I498876\\mri\wmt1.nii,..\t1\3107V10I498876\\mri\mwp1t1.nii,..\t1\3107V10I498876\\mri\smwp1t1.nii,3107V10I498876,79.42
3,3108,V06,04/2013,13.0,15.0,I378222,..\t1raw\3108\MPRAGE_GRAPPA\2013-04-24_10_04_3...,51.8,0,04/2011,-0.153846,25,0,..\t1\3108V06I378222\t1.nii,..\t1\3108V06I378222\,..\t1\3108V06I378222\\mri\wmt1.nii,..\t1\3108V06I378222\\mri\mwp1t1.nii,..\t1\3108V06I378222\\mri\smwp1t1.nii,3108V06I378222,78.86
4,3108,V10,05/2015,22.0,12.0,I498885,..\t1raw\3108\Sag_MPRAGE_GRAPPA\2015-05-06_09_...,53.8,0,04/2011,0.454545,50,1,..\t1\3108V10I498885\t1.nii,..\t1\3108V10I498885\,..\t1\3108V10I498885\\mri\wmt1.nii,..\t1\3108V10I498885\\mri\mwp1t1.nii,..\t1\3108V10I498885\\mri\smwp1t1.nii,3108V10I498885,80.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,101187,V04,06/2022,18.0,7.0,I1616582,..\t1raw\101187\SAG_3D_T1_FSPGR\2022-06-10_12_...,64.0,1,09/2022,0.611111,19,1,..\t1\101187V04I1616582\t1.nii,..\t1\101187V04I1616582\,..\t1\101187V04I1616582\\mri\wmt1.nii,..\t1\101187V04I1616582\\mri\mwp1t1.nii,..\t1\101187V04I1616582\\mri\smwp1t1.nii,101187V04I1616582,60.89
359,101295,V04,08/2022,31.0,23.0,I1616170,"..\t1raw\101295\T1-weighted,_3D_VOLUMETRIC\202...",66.3,1,04/2021,0.258065,25,0,..\t1\101295V04I1616170\t1.nii,..\t1\101295V04I1616170\,..\t1\101295V04I1616170\\mri\wmt1.nii,..\t1\101295V04I1616170\\mri\mwp1t1.nii,..\t1\101295V04I1616170\\mri\smwp1t1.nii,101295V04I1616170,81.80
360,140041,BL,03/2022,22.0,9.0,I1575193,..\t1raw\140041\3D_T1-weighted\2022-04-01_08_5...,55.6,0,02/2022,0.590909,27,1,..\t1\140041BLI1575193\t1.nii,..\t1\140041BLI1575193\,..\t1\140041BLI1575193\\mri\wmt1.nii,..\t1\140041BLI1575193\\mri\mwp1t1.nii,..\t1\140041BLI1575193\\mri\smwp1t1.nii,140041BLI1575193,54.81
361,146875,BL,08/2022,56.0,61.0,I1580986,..\t1raw\146875\3D_T1_MPRAGE\2022-04-21_11_41_...,58.7,1,04/2022,-0.089286,9,0,..\t1\146875BLI1580986\t1.nii,..\t1\146875BLI1580986\,..\t1\146875BLI1580986\\mri\wmt1.nii,..\t1\146875BLI1580986\\mri\mwp1t1.nii,..\t1\146875BLI1580986\\mri\smwp1t1.nii,146875BLI1580986,85.04
